# 04. Memory Agent (Short-term Memory)

간단한 **대화 히스토리**(메모리)를 유지하여 문맥을 반영하는 에이전트를 만듭니다.


In [8]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import AzureOpenAI

# Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview"
)

In [9]:
class MemoryAgent:
    def __init__(self, model: str = None, max_turns: int = 6):
        # Azure OpenAI 배포명 사용
        self.model = model or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
        self.max_turns = max_turns
        self.history = [{"role":"system","content":"You are a helpful assistant with short-term memory."}]
    
    def ask(self, user_text: str) -> str:
        self.history.append({"role":"user","content": user_text})
        # 최근 대화만 유지 (토큰 절약)
        context = [self.history[0]] + self.history[-(self.max_turns+1):]
        completion = client.chat.completions.create(
            model=self.model,
            messages=context,
            max_tokens=200,
            temperature=0.7
        )
        reply = completion.choices[0].message.content.strip()
        self.history.append({"role":"assistant","content": reply})
        return reply

## Try it

In [10]:
agent = MemoryAgent()
print(agent.ask("내 영어 말하기를 2주 안에 올리고 싶은데, 데일리 루틴 만들어줘."))
print(agent.ask("좋아. 그 루틴을 평일/주말로 나눠서 다시 정리해줘."))

영어 말하기 능력을 2주 안에 향상시키기 위한 데일리 루틴을 제안해 드릴게요. 매일 꾸준히 연습하는 것이 중요합니다!

### 1주차: 기초 다지기

**월요일:**
- 30분: 기본 단어 및 표현 공부 (일상 대화에서 자주 쓰는 문장)
- 30분: 짧은 영어 기사 읽기 및 소리 내어 읽기 (뉴스, 블로그 등)

**화요일:**
- 30분: 영어 팟캐스트 듣기 (초급자용)
- 30분: 듣고 요약하기 (듣고 나서 내용 정리)

**수요일:**
- 30분: 영어 회화 앱 사용 (예: HelloTalk, Tandem)
- 30분: 자신에 대해 영어로 소개하기 연습

**목요일:**
- 30분
물론이죠! 아래는 평일과 주말로 나눠서 정리한 데일리 루틴입니다.

### 평일 루틴 (월요일 ~ 금요일)

**월요일:**
- 30분: 기본 단어 및 표현 공부 (일상 대화에서 자주 쓰는 문장)
- 30분: 짧은 영어 기사 읽기 및 소리 내어 읽기

**화요일:**
- 30분: 영어 팟캐스트 듣기 (초급자용)
- 30분: 듣고 요약하기 (내용 정리)

**수요일:**
- 30분: 영어 회화 앱 사용 (예: HelloTalk, Tandem)
- 30분: 자신에 대해 영어로 소개하기 연습

**목요일:**
- 30분: 영어 회화 주제 선정 (예: 여행, 취미 등) 후 관련 표현 공부
- 30분:
물론이죠! 아래는 평일과 주말로 나눠서 정리한 데일리 루틴입니다.

### 평일 루틴 (월요일 ~ 금요일)

**월요일:**
- 30분: 기본 단어 및 표현 공부 (일상 대화에서 자주 쓰는 문장)
- 30분: 짧은 영어 기사 읽기 및 소리 내어 읽기

**화요일:**
- 30분: 영어 팟캐스트 듣기 (초급자용)
- 30분: 듣고 요약하기 (내용 정리)

**수요일:**
- 30분: 영어 회화 앱 사용 (예: HelloTalk, Tandem)
- 30분: 자신에 대해 영어로 소개하기 연습

**목요일:**
- 30분: 영어 회화 주제 선정 (예: 여행, 취미 등) 후 관련 표현 공부
- 3